In [1]:
import torch

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(42)

In [4]:
device = torch.device('cuda')

In [5]:
train_df=pd.read_csv('./fashion-mnist_train.csv')
test_df = pd.read_csv('./fashion-mnist_test.csv')

In [6]:
X_train = train_df.iloc[:,1:].values
y_train = train_df.iloc[:,0].values
X_test = test_df.iloc[:,1:].values
y_test = test_df.iloc[:,0].values

In [7]:
X_train = X_train/255
X_test = X_test/255

In [8]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        super().__init__()
        self.features= torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels,dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index],self.labels[index]

In [9]:
train_dataset = CustomDataset(X_train,y_train)

In [10]:
len(train_dataset)

60000

In [11]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0157, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2431, 0.2392, 0.0824, 0.1137, 0.0902,
         0.2000, 0.5333, 0.2392, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [12]:
test_dataset = CustomDataset(X_test,y_test)

In [13]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [14]:
class MyNN(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64,10)
        )
    def forward(self,x):
        return self.model(x)

In [15]:
learning_rate = 0.1
epochs = 100

In [16]:
model = MyNN(X_train.shape[1])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,weight_decay=1e-4)

In [17]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features,batch_labels in train_loader:
        batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
        
        outputs = model(batch_features)
        
        loss = criterion(outputs,batch_labels)
        
        total_epoch_loss+=loss.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {total_epoch_loss/len(train_loader)}")

Epoch: 1, Loss: 0.606932546766599
Epoch: 2, Loss: 0.4839596184651057
Epoch: 3, Loss: 0.4471514514525731
Epoch: 4, Loss: 0.42515789304176965
Epoch: 5, Loss: 0.4073649721264839
Epoch: 6, Loss: 0.39402178866465887
Epoch: 7, Loss: 0.3839616592844327
Epoch: 8, Loss: 0.3779019603610039
Epoch: 9, Loss: 0.3684530872484048
Epoch: 10, Loss: 0.36589099501371386
Epoch: 11, Loss: 0.36172818118333816
Epoch: 12, Loss: 0.3533182066321373
Epoch: 13, Loss: 0.34808402404785155
Epoch: 14, Loss: 0.34396871750752134
Epoch: 15, Loss: 0.34315298453569415
Epoch: 16, Loss: 0.3384735964258512
Epoch: 17, Loss: 0.33631830992102624
Epoch: 18, Loss: 0.33521430836518606
Epoch: 19, Loss: 0.3286382370471954
Epoch: 20, Loss: 0.32768861591418585
Epoch: 21, Loss: 0.3265842638731003
Epoch: 22, Loss: 0.3208629125873248
Epoch: 23, Loss: 0.3208887868603071
Epoch: 24, Loss: 0.3220463091591994
Epoch: 25, Loss: 0.3169858889957269
Epoch: 26, Loss: 0.31907777786652247
Epoch: 27, Loss: 0.3122964305837949
Epoch: 28, Loss: 0.31124084

KeyboardInterrupt: 

In [18]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [19]:
total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
        
        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)
        total+=batch_labels.shape[0]
        correct+=(predicted==batch_labels).sum().item()
        
print(correct/total)

0.8924


In [20]:
total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in train_loader:
        batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)
        
        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)
        total+=batch_labels.shape[0]
        correct+=(predicted==batch_labels).sum().item()
        
print(correct/total)

0.9281833333333334
